# Row-Oriented Programming

A fundamental unit of computation in `datajoint` pipelines is a table row. `djutils` provides handy tools for row-oriented programming with `rowproperty`, `cache_rowproperty`, and `keys`.


### `rowproperty`

This is a decorator for methods of datajoint `tables` and djutils `keys`. It acts in a similar fashion to the python `property` decorator, but beyond that it serves 2 major purposes:


1) rowproperties can only be accessed when the table or key object (`self`) is a single `row`
    - An error is thrown if `self` is not a single `row`.
    - This promotes code safety and readability:
        - The code writer can be assured that the `self` is a row (imperative for functions like `self.fetch1` in tables and `self.item` in keys).
        - The code reader understands that the `self` within the rowproproperty definition refers to a single row of the table or key object.
2) rowproperties can be cached with the `cache_rowproperty` context manager


### `cache_rowproperty`

This is a context manager that caches rowproperties in memory the first time they are computed. Within this context, rowproperties are only computed once and retrieved from the cache on subsequent calls.


### `keys`

This is a decorator for classes. This decorator turns the class into a pseudo-table containing the product of primary keys from datajoint tables (can be virtual). This decorator enables row-oriented programming with `rowproperty` and `cache_rowproperty` without having to explicitly create and store tables on the database.

# Walkthrough

Let's create a `Scale` lookup table filled that stores variables for scaling data.

We'll also define a `transform` method that scales input data by the `scale` value for that row.
`transform` will be decorated by `rowproperty` from djutils.

In [1]:
from djutils import Schema, rowproperty

schema = Schema("djutils_tutorial_1")


@schema.lookup
class Scale:
    definition = """
    scale : decimal(6, 3)
    """
    contents = [
        [0.5], [1.0], [1.5],
    ]
    
    @rowproperty
    def transform(self):
        scale = float(self.fetch1("scale"))
        print(f"Loading transform   x -> x * {scale}")
        return lambda x: x * scale

Connecting ewang@at-database.ad.bcm.edu:3306


Let's take a look at the `Scale` table that we created, which has 3 rows from `contents`

In [2]:
Scale()

scale
0.500
1.000
1.500


Let's restrict the `Scale` table to the row where scale = 1.5

In [3]:
row = Scale & "scale=1.5"
row

scale
1.500


Let's access the `transform` rowproperty and use it to transform a couple of numbers (2.0 and 3.0)

In [4]:
transform = row.transform

for x in [2.0, 3.0]:
    y = transform(x)
    print(f"{x} -> {y}")

Loading transform   x -> x * 1.5
2.0 -> 3.0
3.0 -> 4.5


Notice how if we try to access `transform` when `Scale` is not restricted to a single row, we will get a djutils `RestrictionError`

In [5]:
rows = Scale & "scale in (1, 1.5)"
display(rows)

print("Trying to access transform")
transform = rows.transform

scale
1.000
1.500


Trying to access transform


RestrictionError: Table must be restricted to single row.

---
Now, let's create a lookup table `X` for the variable `x`

And a computed table `Y` scales `x` using the rowproperty `transform` and stores the result as `y`

In [6]:
@schema.lookup
class X:
    definition = """
    x : float
    """
    contents = [
        [1.0], [2.0], [3.0], [4.0],
    ]
    
@schema.computed
class Y:
    definition = """
    -> X
    -> Scale
    ---
    y : float
    """
    
    def make(self, key):
        key["y"] = (Scale & key).transform(key["x"])
        self.insert1(key)

Let's populate `Y` using the scaling factor 0.5

In [7]:
Y.populate('scale = 0.5')

Y & 'scale = 0.5'

Loading transform   x -> x * 0.5
Loading transform   x -> x * 0.5
Loading transform   x -> x * 0.5
Loading transform   x -> x * 0.5


x,scale,y
1.0,0.500,0.5
2.0,0.500,1.0
3.0,0.500,1.5
4.0,0.500,2.0


Notice how, during the population of the 4 rows, the same transform `x * 0.5` was loaded 4 times in the `make` function of `Y`.

---

Can we avoid this unnecessary computation?

Solution : `cache_rowproperty`

Within the context of `cache_rowproperty`, rowproperties are stored in memory and only need to be computed once.

Using `cache_rowproperty`, let's populate `Y` using the scaling factor 1.5

In [8]:
from djutils import cache_rowproperty

with cache_rowproperty():
    Y.populate('scale = 1.5')
    
Y & 'scale = 1.5'

Loading transform   x -> x * 1.5


x,scale,y
1.0,1.500,1.5
2.0,1.500,3.0
3.0,1.500,4.5
4.0,1.500,6.0


Notice how trasform was only loaded once, despite it being accessed 4 times in the `make` function of `Y`

----

Now, let's say that you want to compose the scaling transformation with an offset variable, stored in an `Offset` table

In [9]:
@schema.lookup
class Offset:
    definition = """
    offset : decimal(6, 3)
    """
    contents = [
        [0.0], [0.1], [0.2],
    ]

One option would be to create a `ScaleOffsetTable` lookup table and insert all of the keys from `Scale` and `Offset` into `ScaleOffsetTable`, and define a `transform` rowproperty that uses the `scale` and `offset` variables.

In [10]:
@schema.lookup
class ScaleOffsetTable:
    definition = """
    -> Scale
    -> Offset
    """
    
    @property
    def contents(self):
        return Scale * Offset
    
    @rowproperty
    def transform(self):
        scale, offset = map(float, (self.fetch1("scale", "offset")))
        print(f"Loading transform   x -> x * {scale} + {offset}")
        return lambda x: x * scale + offset

We can see that our table has 9 rows (3 scale x 3 offset)

In [11]:
ScaleOffsetTable()

scale,offset
0.500,0.000
1.000,0.000
1.500,0.000
0.500,0.100
1.000,0.100
1.500,0.100
0.500,0.200
1.000,0.200
1.500,0.200


And we can use the `transform` rowproperty of `ScaleOffsetTable`

In [12]:
transform = (ScaleOffsetTable & 'scale=1 and offset=0.1').transform
x = 4.0
y = transform(x)
print(f"{x} -> {y}")

Loading transform   x -> x * 1.0 + 0.1
4.0 -> 4.1


But isn't it wasteful and unnecessary to create the table `ScaleOffsetTable`, which is just the product of `Scale` and `Offset`?

How can we use the programming approach of `rowproperty` and `cache_rowproperty` without wasting database storage on the product of tables?

---

Solution: `keys`

This is a decorator that turns a class into a table-like object.

The decorated class must define a `keys` property that returns a list of tables (can be virtual). The decorator will join the primary keys of those tables, which produces a set of rows for the user to work with.

We will also define a `transform` rowproperty that performs the same function as the previous example. However, unlike the previous example where we used up database storage to create and fill a table with the 9 rows, `keys` enables users to perform class-based, row-oriented programming without having to create and fill tables on the database.

In [13]:
from djutils import keys

@keys
class ScaleOffsetKey:
    
    @property
    def keys(self):
        return [Scale, Offset]
    
    @rowproperty
    def transform(self):
        scale = float(self.item["scale"])
        offset = float(self.item["offset"])
        print(f"Loading transform   x -> x * {scale} + {offset}")
        return lambda x: x * scale + offset

In [14]:
ScaleOffsetKey()

9 x ['scale', 'offset']

`primary_key` gives you the primary keys of the joined tables

In [15]:
ScaleOffsetKey.primary_key

['scale', 'offset']

The decorated class can be restricted just like a normal datajoint table

In [16]:
ScaleOffsetKey & 'scale = 0.5'

3 x ['scale', 'offset']

`key` gives you the underlying table product

In [17]:
(ScaleOffsetKey & 'scale = 0.5').key

scale,offset
0.500,0.000
0.500,0.100
0.500,0.200


When restricted to a single row, `item` gives you a dictionary of the row

In [18]:
(ScaleOffsetKey & 'scale = 0.5 and offset = 0.1').item

{'scale': Decimal('0.500'), 'offset': Decimal('0.100')}

And just like normal tables, `rowproperty` and `cache_rowproperty` will work for classes decorated with `keys`.

In [19]:
print("\n------- Transforming WITHOUT cacheing-------\n")
for x in range(3):
    y = (ScaleOffset & 'scale = 0.5 and offset = 0.1').transform(x)
    print(f"{x} -> {y}")
    
    
print("\n------- Transforming WITH cacheing-------\n")
with cache_rowproperty():
    for x in range(3):
        y = (ScaleOffset & 'scale = 0.5 and offset = 0.1').transform(x)
        print(f"{x} -> {y}")


------- Transforming WITHOUT cacheing-------

Loading transform   x -> x * 0.5 + 0.1
0 -> 0.1
Loading transform   x -> x * 0.5 + 0.1
1 -> 0.6
Loading transform   x -> x * 0.5 + 0.1
2 -> 1.1

------- Transforming WITH cacheing-------

Loading transform   x -> x * 0.5 + 0.1
0 -> 0.1
1 -> 0.6
2 -> 1.1
